In [ ]:
from core.utils import Tibanna
from core import ff_utils


# for a given experiment set and some parameters like instrument
# print set of files and their partA hic workflow status
# if there are one that are running report the number of running cases
# if there are file pairs that don't have a corresponding part A, report them separately

env = 'fourfront-webdev'
tibanna = Tibanna(env=env)

file_not_found = 0
deleted = 0
success = 0
patch_problem = 0
ff = ff_utils.fdn_connection(key=tibanna.ff_keys)
files_resp = ff_utils.get_metadata('files', connection=ff)['@graph']
print len(files_resp), 'files'
no_size = 0
for a_file in files_resp:
    # check if there is a filename but no filesize
    if a_file.get('file_size'):
        print a_file['accession']
    if a_file.get('filename') and not a_file.get('file_size'):
        # decide on the bucket
        if 'FileProcessed' in a_file['@type']:
            bucket = tibanna.s3.outfile_bucket
        else:
            bucket = tibanna.s3.raw_file_bucket
            
        # check if file is in s3
        head_info = tibanna.s3.does_key_exist(a_file['upload_key'], bucket)
        if not head_info:
            file_not_found += 1
            continue
        file_size = head_info['ContentLength']
        success += 1
        if a_file['status'] == 'deleted':
            deleted += 1

        patch_data = {'file_size': file_size}
        try:
            ff_utils.patch_metadata(patch_data, obj_id=a_file['uuid'] ,connection=ff)
            success += 1
        except Exception as e:
            print e
            patch_problem += 1

print file_not_found, 'files missing from s3'
print patch_problem, 'files had patch problems'
print deleted, 'deleted files patched with filesize data'
print success, 'total files patched'

In [ ]:
from core.utils import Tibanna
from core import ff_utils


# for a given experiment set and some parameters like instrument
# print set of files and their partA hic workflow status
# if there are one that are running report the number of running cases
# if there are file pairs that don't have a corresponding part A, report them separately

env = 'fourfront-webdev'
tibanna = Tibanna(env=env)

file_not_found = 0
deleted = 0
success = 0
patch_problem = 0
ff = ff_utils.fdn_connection(key=tibanna.ff_keys)
files_resp = ff_utils.get_metadata('files', connection=ff)['@graph']
print len(files_resp), 'files'

raw_problem = 0
object_problem = 0
embedded_problem = 0

counter = 0
for a_file in files_resp:
    counter += 1
    file_uuid = a_file['uuid']
    try:
        file_resp = ff_utils.get_metadata(file_uuid, connection=ff, frame = "raw")
    except:
        raw_problem += 1
        
    try:
        file_resp = ff_utils.get_metadata(file_uuid, connection=ff, frame = "object")
    except:
        object_problem += 1
        
    try:
        file_resp = ff_utils.get_metadata(file_uuid, connection=ff, frame = "embedded")
    except:
        embedded_problem += 1
    if counter % 100 == 0:
        print counter, raw_problem, object_problem, embedded_problem

print raw_problem
print object_problem
print embedded_problem

        